In [ ]:
from collections import OrderedDict

class LRUCache:
    def __init__(self, capacity: int):
        self.cap = capacity
        self.od = OrderedDict()

    def get(self, key: int) -> int:
        if key not in self.od:
            return -1
        self.od.move_to_end(key)         # mark as recently used
        return self.od[key]

    def put(self, key: int, value: int) -> None:
        if key in self.od:
            self.od.move_to_end(key)     # will update recency
        self.od[key] = value

        if len(self.od) > self.cap:
            self.od.popitem(last=False)  # evict least recently used


In [17]:
import platform
platform.python_version()

'3.11.13'

In [6]:
from typing import Optional
class Node:
    __slots__ = ("key", "value", "next", "prev")
    def __init__(self, key:int, value:int = 0, prev: Optional["Node"] = None , next: Optional["Node"]= None):
        self.key = key
        self.value = value
        self.next = next
        self.prev = prev

class LRUCache:
    def __init__(self, cap: int):
        self.left = Node(0) #LRU sentinel
        self.right = Node(0) #MRU sentinel
        self.left.next = self.right
        self.right.prev = self.left
        self.cap = cap
        self.cache : dict[int, Node]= {}

    def get(self, key: int) -> int:
        if key in self.cache:
            node = self.cache[key]
            self._move_to_right(node)
            return node.value
        return -1

    def put(self, key: int, val: int) -> None:
        if key in self.cache:
            node = self.cache[key]
            node.value = val
            self._move_to_right(node)
            return
        
        if len(self.cache) >= self.cap:
            self._evict()
        
        node = Node(key, val)
        self.cache[key] = node
        self._insert_at_right(node)
    
    def _move_to_right(self, node: Node) -> None:
        self._remove(node)
        self._insert_at_right(node)

    def _remove(self, node: Node) -> None:
        prev, next = node.prev, node.next
        prev.next, next.prev = next, prev

    def _insert_at_right(self, node: Node) -> None:
        end = self.right.prev
        end.next = node
        node.prev = end
        node.next = self.right
        self.right.prev = node

    def _evict(self):
        lru = self.left.next
        self._remove(lru)
        self.cache.pop(lru.key)
    
    def __repr__(self):
        ptr = self.left
        ret = []
        while ptr:
            ret.append(f"[{ptr.key}:{ptr.value}]")
            ptr = ptr.next
        return "<->".join(ret)

lru = LRUCache(5)
lru.put(1,1)
lru.put(2,2)
lru.put(3,3)
print(lru.get(1))
print(lru)
lru.put(4,4)
lru.put(5,5)
print(lru)
lru.put(6,6)
print(lru)


1
[0:0]<->[2:2]<->[3:3]<->[1:1]<->[0:0]
[0:0]<->[2:2]<->[3:3]<->[1:1]<->[4:4]<->[5:5]<->[0:0]
[0:0]<->[3:3]<->[1:1]<->[4:4]<->[5:5]<->[6:6]<->[0:0]


In [ ]:
from __future__ import annotations
from dataclasses import dataclass, field

@dataclass(eq=False, repr=False)
class Node:
    key: int
    value: int = 0
    prev: Node | None = None
    next: Node | None = None


@dataclass
class LRUCache:
    capacity: int
    left: Node = field(default_factory=lambda: Node(0))   # LRU sentinel
    right: Node = field(default_factory=lambda: Node(0))  # MRU sentinel
    cache: dict[int, Node] = field(default_factory=dict)

    def __post_init__(self) -> None:
        self.left.next = self.right
        self.right.prev = self.left

Node(key=1, value=1, prev=None, next=Node(key=2, value=1, prev=..., next=None))
